# 整合minio中的数据读写

## era5搜索和读取

In [1]:
from hydro_opendata.data.minio import Era5L
import numpy as np
import geopandas as gpd

In [2]:
era5 = Era5L('wis')    # wis为数据集名称，wis为国内范围，后期加入camels数据集范围

- 主要通过catalog和reader两个属性对象进行数据操作

In [3]:
# dir(era5.catalog), dir(era5.reader)

print(era5.catalog.spatial_resolution)
print(era5.catalog.start_time)
print(era5.catalog.end_time)

0.1 x 0.1; Native resolution is 9 km.
2015-07-01T00:00:00
2021-12-31T23:00:00


In [4]:
aoi = gpd.read_file('basin.geojson')
aoi

,id,geometry
0,0,"MULTIPOLYGON (((122.44241 39.80139, 122.39342 ..."


- 获取数据时间范围和空间范围

In [5]:
e = era5.catalog.search(aoi=aoi)
e

,id,start_time,end_time,geometry
0,era5-land,2015-07-01,2021-12-31 23:00:00,"POLYGON ((122.39342 39.81027, 122.31744 39.833..."


- 通过指定四至范围读取

In [6]:
bbox=(121,39,123,40)
start_time=np.datetime64("2021-06-01T00:00:00.000000000")
end_time=np.datetime64("2021-06-30T23:00:00.000000000")

e1 = era5.reader.open_dataset(data_variables=['Total precipitation'], start_time=start_time, end_time=end_time, bbox=bbox)
e1

/home/zhujianfeng/.conda/envs/dev-env/lib/python3.10/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:  (time: 720, lon: 21, lat: 11)
Coordinates:
  * lat      (lat) float32 39.0 39.1 39.2 39.3 39.4 ... 39.6 39.7 39.8 39.9 40.0
  * lon      (lon) float32 121.0 121.1 121.2 121.3 ... 122.7 122.8 122.9 123.0
  * time     (time) datetime64[ns] 2021-06-01 ... 2021-06-30T23:00:00
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(24, 21, 11), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

- 通过矢量数据文件读取

In [7]:
shp = 'basin.geojson'
e2 = era5.reader.from_shp(data_variables=['Total precipitation'], start_time=start_time, end_time=end_time, shp=shp)
e2

<xarray.Dataset>
Dimensions:  (time: 720, lon: 6, lat: 5)
Coordinates:
  * lat      (lat) float32 39.8 39.9 40.0 40.1 40.2
  * lon      (lon) float32 122.3 122.4 122.5 122.6 122.7 122.8
  * time     (time) datetime64[ns] 2021-06-01 ... 2021-06-30T23:00:00
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(24, 6, 5), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

- 通过已有aoi对象读取

In [8]:
e3 = era5.reader.from_aoi(data_variables=['Total precipitation'], start_time=start_time, end_time=end_time, aoi=aoi)
e3

<xarray.Dataset>
Dimensions:  (time: 720, lon: 6, lat: 5)
Coordinates:
  * lat      (lat) float32 39.8 39.9 40.0 40.1 40.2
  * lon      (lon) float32 122.3 122.4 122.5 122.6 122.7 122.8
  * time     (time) datetime64[ns] 2021-06-01 ... 2021-06-30T23:00:00
Data variables:
    tp       (time, lon, lat) float32 dask.array<chunksize=(24, 6, 5), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-21 12:13:40 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## gpm搜索和读取

In [9]:
from hydro_opendata.data.minio import GPM

In [10]:
gpm = GPM('wis')    # wis为数据集名称，wis为国内范围，camels为camels数据集范围

- 主要通过catalog和reader两个属性对象进行数据操作

In [11]:
# dir(gpm.catalog), dir(gpm.reader)

print(gpm.catalog.temporal_resolution)
print(gpm.catalog.start_time)
print(gpm.catalog.end_time)

half-hourly
2016-01-01T00:00:00.000000000
2023-09-05T05:30:00.000000000


- 获取数据时间范围和空间范围

In [12]:
g = gpm.catalog.search(aoi=aoi)
g

,id,start_time,end_time,geometry
0,gpm-imerg-early,2016-01-01,2023-09-05 05:30:00,"POLYGON ((122.39342 39.81027, 122.31744 39.833..."


- 通过指定四至范围读取

In [13]:
bbox=(121,39,123,40)
start_time=np.datetime64("2023-06-01T00:00:00.000000000")
end_time=np.datetime64("2023-06-30T23:30:00.000000000")

g1 = gpm.reader.open_dataset(start_time=start_time, end_time=end_time, bbox=bbox)
g1

<xarray.DataArray 'precipitationCal' (time: 1440, lon: 20, lat: 10)>
dask.array<getitem, shape=(1440, 20, 10), dtype=float32, chunksize=(48, 20, 10), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 39.05 39.15 39.25 39.35 ... 39.65 39.75 39.85 39.95
  * lon      (lon) float32 121.1 121.2 121.2 121.3 ... 122.7 122.8 122.8 122.9
  * time     (time) datetime64[ns] 2023-06-01 ... 2023-06-30T23:30:00
Attributes:
    CodeMissingValue:  -9999.9
    DimensionNames:    time,lon,lat
    Units:             mm/hr
    units:             mm/hr

- 通过矢量数据文件读取

In [14]:
shp = 'basin.geojson'
g2 = gpm.reader.from_shp(start_time=start_time, end_time=end_time, shp=shp)
g2

<xarray.DataArray 'precipitationCal' (time: 1440, lon: 5, lat: 4)>
dask.array<getitem, shape=(1440, 5, 4), dtype=float32, chunksize=(48, 5, 4), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 39.85 39.95 40.05 40.15
  * lon      (lon) float32 122.3 122.4 122.6 122.7 122.8
  * time     (time) datetime64[ns] 2023-06-01 ... 2023-06-30T23:30:00
Attributes:
    CodeMissingValue:  -9999.9
    DimensionNames:    time,lon,lat
    Units:             mm/hr
    units:             mm/hr

- 通过已有aoi对象读取

In [15]:
aoi = gpd.read_file(shp)
g3 = gpm.reader.from_aoi(start_time=start_time, end_time=end_time, aoi=aoi)
g3

<xarray.DataArray 'precipitationCal' (time: 1440, lon: 5, lat: 4)>
dask.array<getitem, shape=(1440, 5, 4), dtype=float32, chunksize=(48, 5, 4), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 39.85 39.95 40.05 40.15
  * lon      (lon) float32 122.3 122.4 122.6 122.7 122.8
  * time     (time) datetime64[ns] 2023-06-01 ... 2023-06-30T23:30:00
Attributes:
    CodeMissingValue:  -9999.9
    DimensionNames:    time,lon,lat
    Units:             mm/hr
    units:             mm/hr

## gfs搜索和读取

In [16]:
from hydro_opendata.data.minio import GFS

In [17]:
gfs = GFS('wis', 'tp')    # wis为数据集名称，wis为国内范围，camels为camels数据集范围

- 主要通过catalog和reader两个属性对象进行数据操作

In [18]:
# dir(gfs.catalog), dir(gfs.reader)

print(gfs.catalog.data_sources)
print(gfs.catalog.start_time)
print(gfs.catalog.end_time)

NOAA
2016-07-10T00
2023-09-05T06


- 获取数据时间范围和空间范围

In [19]:
f = gfs.catalog.search(aoi=aoi)
f

,id,start_time,end_time,geometry
1,gfs_atmos.tp,2022-09-01,2023-09-05 06:00:00,"POLYGON ((122.39342 39.81027, 122.31744 39.833..."
0,gfs_atmos.tp,2016-07-10,2022-08-31 18:00:00,"POLYGON ((122.39342 39.81027, 122.31744 39.833..."


- 通过指定四至范围读取

In [20]:
bbox=(121,39,123,40)
creation_date=np.datetime64("2023-06-01")

f1 = gfs.reader.open_dataset(creation_date=creation_date, creation_time='00', bbox=bbox)
f1

<xarray.Dataset>
Dimensions:     (lat: 5, lon: 9, step: 120, time: 1, valid_time: 120)
Coordinates:
  * lat         (lat) float64 39.0 39.25 39.5 39.75 40.0
  * lon         (lon) float64 121.0 121.2 121.5 121.8 ... 122.5 122.8 123.0
  * step        (step) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * time        (time) datetime64[ns] 2023-06-01
  * valid_time  (valid_time) datetime64[ns] 2023-06-01T01:00:00 ... 2023-06-06
Data variables:
    tp          (valid_time, step, lat, lon) float64 dask.array<chunksize=(24, 1, 5, 9), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0

- 通过矢量数据文件读取

In [21]:
shp = 'basin.geojson'
f2 = gfs.reader.from_shp(creation_date=creation_date, creation_time='00', shp=shp)
f2

<xarray.Dataset>
Dimensions:     (lat: 3, lon: 3, step: 120, time: 1, valid_time: 120)
Coordinates:
  * lat         (lat) float64 39.75 40.0 40.25
  * lon         (lon) float64 122.2 122.5 122.8
  * step        (step) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * time        (time) datetime64[ns] 2023-06-01
  * valid_time  (valid_time) datetime64[ns] 2023-06-01T01:00:00 ... 2023-06-06
Data variables:
    tp          (valid_time, step, lat, lon) float64 dask.array<chunksize=(24, 1, 3, 3), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0

- 通过已有aoi对象读取

In [22]:
aoi = gpd.read_file(shp)
f3 = gfs.reader.from_aoi(creation_date=creation_date, creation_time='00', aoi=aoi)
f3

<xarray.Dataset>
Dimensions:     (lat: 3, lon: 3, step: 120, time: 1, valid_time: 120)
Coordinates:
  * lat         (lat) float64 39.75 40.0 40.25
  * lon         (lon) float64 122.2 122.5 122.8
  * step        (step) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * time        (time) datetime64[ns] 2023-06-01
  * valid_time  (valid_time) datetime64[ns] 2023-06-01T01:00:00 ... 2023-06-06
Data variables:
    tp          (valid_time, step, lat, lon) float64 dask.array<chunksize=(24, 1, 3, 3), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0